<a href="https://colab.research.google.com/github/atishay01/Ensemble-Driven-Multimodal-Fake-News-Detection-System/blob/main/Multimodal_FakeNews_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch torchvision pandas numpy matplotlib scikit-learn

from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
from PIL import Image
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from transformers import BertTokenizer, BertModel
import torch

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Fake News Model/BBC_sample.csv')

df.head()


,title,pubDate,guid,link,description
0,Ukraine: Angry Zelensky vows to punish Russian...,"Mon, 07 Mar 2022 08:01:56 GMT",https://www.bbc.co.uk/news/world-europe-60638042,https://www.bbc.co.uk/news/world-europe-606380...,The Ukrainian president says the country will ...
1,War in Ukraine: Taking cover in a town under a...,"Sun, 06 Mar 2022 22:49:58 GMT",https://www.bbc.co.uk/news/world-europe-60641873,https://www.bbc.co.uk/news/world-europe-606418...,"Jeremy Bowen was on the frontline in Irpin, as..."
2,Ukraine war 'catastrophic for global food',"Mon, 07 Mar 2022 00:14:42 GMT",https://www.bbc.co.uk/news/business-60623941,https://www.bbc.co.uk/news/business-60623941?a...,One of the world's biggest fertiliser firms sa...
3,Manchester Arena bombing: Saffie Roussos's par...,"Mon, 07 Mar 2022 00:05:40 GMT",https://www.bbc.co.uk/news/uk-60579079,https://www.bbc.co.uk/news/uk-60579079?at_medi...,The parents of the Manchester Arena bombing's ...
4,Ukraine conflict: Oil price soars to highest l...,"Mon, 07 Mar 2022 08:15:53 GMT",https://www.bbc.co.uk/news/business-60642786,https://www.bbc.co.uk/news/business-60642786?a...,Consumers are feeling the impact of higher ene...


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Data Preprocessing Function

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    return ' '.join(tokens)

df['processed_title'] = df['title'].apply(preprocess_text)
df['processed_description'] = df['description'].apply(preprocess_text)
df[['processed_title', 'processed_description']].head()


,processed_title,processed_description
0,ukraine angry zelensky vows punish russian atr...,ukrainian president says country forgive forge...
1,war ukraine taking cover town attack,jeremy bowen frontline irpin residents came ru...
2,ukraine war catastrophic global food,one world biggest fertiliser firms says confli...
3,manchester arena bombing saffie roussos parent...,parents manchester arena bombing youngest vict...
4,ukraine conflict oil price soars highest level...,consumers feeling impact higher energy costs f...


In [ ]:
# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :]  # [CLS] token embedding

# Apply the BERT embedding
df['title_embedding'] = df['processed_title'].apply(lambda x: get_bert_embedding(x).squeeze().numpy())
df['description_embedding'] = df['processed_description'].apply(lambda x: get_bert_embedding(x).squeeze().numpy())

df['title_embedding'].head()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

,title_embedding
0,"[-0.20370954, 0.33402073, -0.15621713, -0.1395..."
1,"[-0.6362953, -0.010361542, -0.10196573, -0.181..."
2,"[-0.70683473, 0.040736496, -0.289261, -0.27386..."
3,"[-0.18663417, -0.17270315, -0.4284545, 0.17314..."
4,"[-1.0233779, 0.064748704, 0.18019518, -0.36657..."


In [ ]:
# Function for Loading the Images

#import requests
#from PIL import Image
#import os

# Function to download images
#def download_image(url, save_path):
#    try:
#        img_data = requests.get(url).content
#        with open(save_path, 'wb') as handler:
#            handler.write(img_data)
#    except Exception as e:
#        print(f"Error downloading image: {e}")

# Create a directory to save images
#os.makedirs('images', exist_ok=True)

# Download images based on the URLs in your DataFrame
#df['image_path'] = df['link'].apply(lambda x: f"images/{x.split('/')[-1]}.jpg")
#df['link'].apply(lambda x: download_image(x, f"images/{x.split('/')[-1]}.jpg"))


In [ ]:
#Extraction of Images from webpage

!pip install beautifulsoup4 requests

import requests
from bs4 import BeautifulSoup
import os


In [ ]:
def get_image_url_from_webpage(url):
    try:
        response = requests.get(url)

        soup = BeautifulSoup(response.content, 'html.parser')

        img_tag = soup.find('img')
        if img_tag and 'src' in img_tag.attrs:
            img_url = img_tag['src']

            if not img_url.startswith('http'):
                img_url = requests.compat.urljoin(url, img_url)
            return img_url
        else:
            return None
    except Exception as e:
        print(f"Error fetching image URL: {e}")
        return None

df['image_url'] = df['link'].apply(get_image_url_from_webpage)
df[['link', 'image_url']].head()


,link,image_url
0,https://www.bbc.co.uk/news/world-europe-606380...,https://ichef.bbci.co.uk/ace/standard/976/cpsp...
1,https://www.bbc.co.uk/news/world-europe-606418...,https://ichef.bbci.co.uk/ace/standard/400/cpsp...
2,https://www.bbc.co.uk/news/business-60623941?a...,https://ichef.bbci.co.uk/ace/standard/976/cpsp...
3,https://www.bbc.co.uk/news/uk-60579079?at_medi...,https://ichef.bbci.co.uk/ace/standard/1504/cps...
4,https://www.bbc.co.uk/news/business-60642786?a...,https://ichef.bbci.co.uk/ace/standard/976/cpsp...


In [ ]:
import os
import requests

os.makedirs('images', exist_ok=True)
def download_image_from_url(image_url, save_path):
    try:
        img_data = requests.get(image_url).content
        with open(save_path, 'wb') as handler:
            handler.write(img_data)
        return True
    except Exception as e:
        print(f"Error downloading image: {e}")
        return False

df['image_path'] = df['image_url'].apply(lambda x: f"images/{x.split('/')[-1]}" if x else None)
df['download_successful'] = df.apply(lambda row: download_image_from_url(row['image_url'], row['image_path']) if row['image_url'] else False, axis=1)

df[['link', 'image_url', 'image_path', 'download_successful']].head()


,link,image_url,image_path,download_successful
0,https://www.bbc.co.uk/news/world-europe-606380...,https://ichef.bbci.co.uk/ace/standard/976/cpsp...,images/_123565703_de16-1.jpg,True
1,https://www.bbc.co.uk/news/world-europe-606418...,https://ichef.bbci.co.uk/ace/standard/400/cpsp...,images/_123575296_gettyimages-1238994406.jpg,True
2,https://www.bbc.co.uk/news/business-60623941?a...,https://ichef.bbci.co.uk/ace/standard/976/cpsp...,images/_123565445_gettyimages-1238855471.jpg,True
3,https://www.bbc.co.uk/news/uk-60579079?at_medi...,https://ichef.bbci.co.uk/ace/standard/1504/cps...,images/_123471160_2hi040414281.jpg,True
4,https://www.bbc.co.uk/news/business-60642786?a...,https://ichef.bbci.co.uk/ace/standard/976/cpsp...,images/_123574207_gettyimages-1161679863.jpg,True


In [ ]:
resnet = models.resnet50(pretrained=True)
resnet.eval()

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def get_image_embedding(image_path):
    try:
        image = Image.open(image_path)
        image = preprocess(image).unsqueeze(0)
        with torch.no_grad():
            image_embedding = resnet(image)
        return image_embedding.squeeze().numpy()
    except Exception as e:
        print(f"Error processing image: {e}")
        return None

df['image_embedding'] = df['image_path'].apply(get_image_embedding)

df[['image_embedding']].head()


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 147MB/s]


Error processing image: The size of tensor a (4) must match the size of tensor b (3) at non-singleton dimension 0
Error processing image: The size of tensor a (4) must match the size of tensor b (3) at non-singleton dimension 0
Error processing image: output with shape [1, 224, 224] doesn't match the broadcast shape [3, 224, 224]
Error processing image: The size of tensor a (4) must match the size of tensor b (3) at non-singleton dimension 0
Error processing image: The size of tensor a (4) must match the size of tensor b (3) at non-singleton dimension 0
Error processing image: The size of tensor a (4) must match the size of tensor b (3) at non-singleton dimension 0
Error processing image: 'NoneType' object has no attribute 'read'


,image_embedding
0,"[2.4157312, 0.03873287, -1.8824877, -2.6463547..."
1,"[0.8564257, -2.5707176, 1.82571, 0.20114884, -..."
2,"[-0.48058164, 1.8887206, -1.6726205, -2.313653..."
3,"[3.2459917, -0.09845278, -0.623371, -2.4864655..."
4,"[-2.9479365, -1.9584193, 0.7021371, 0.5380189,..."


In [ ]:
print(df.keys())

Index(['title', 'pubDate', 'guid', 'link', 'description', 'processed_title',
       'processed_description', 'title_embedding', 'description_embedding',
       'image_url', 'image_path', 'download_successful', 'image_embedding',
       'text_embedding', 'combined_embedding'],
      dtype='object')


In [ ]:
missing_embeddings = df['image_embedding'].isnull().sum()

#print(f"Number of missing embeddings: {missing_embeddings}")


Replace Missing Embeddings with 0
Assuming the embedding size from ResNet-50 is 2048

In [ ]:
df['text_embedding'] = df.apply(lambda row: np.concatenate((row['title_embedding'], row['description_embedding'])), axis=1)

embedding_length = len(df['description_embedding'].iloc[0])

df['image_embedding'] = df['image_embedding'].apply(
    lambda x: x if x is not None else np.zeros(embedding_length)
)

# Concatenate
df['combined_embedding'] = df.apply(
    lambda row: np.concatenate((row['description_embedding'], row['image_embedding'])),
    axis=1
)


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
print(df.columns)

Index(['title', 'pubDate', 'guid', 'link', 'description', 'processed_title',
       'processed_description', 'title_embedding', 'description_embedding',
       'image_url', 'image_path', 'download_successful', 'image_embedding',
       'text_embedding', 'combined_embedding'],
      dtype='object')


In [ ]:
data_dir = "/content/drive/MyDrive/Fake News Model/train"

# ***Image Model***

In [ ]:
# Transformations to standardize image input
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Generate Labels
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

In [ ]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

from torchvision.models import resnet50, ResNet50_Weights

# Load the ResNet50 (Default)
model = resnet50(weights=ResNet50_Weights.DEFAULT)

model.fc = nn.Linear(model.fc.in_features, 2) #Binary Classification

#migrate to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 126MB/s]


In [ ]:
import torch.optim as optim

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


*Training Image Model*

In [ ]:
num_epochs = 10  # Set the number of epochs

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")


Epoch 1/10, Loss: 0.2689437593292931
Epoch 2/10, Loss: 0.1760978494649348
Epoch 3/10, Loss: 0.1414341028455807
Epoch 4/10, Loss: 0.08395123205643715
Epoch 5/10, Loss: 0.05408811402183188
Epoch 6/10, Loss: 0.061686407270431315
Epoch 7/10, Loss: 0.0504176368362412
Epoch 8/10, Loss: 0.17544084720556502
Epoch 9/10, Loss: 0.2138516344939885
Epoch 10/10, Loss: 0.05033698801518134


*Load the Model*

In [ ]:
model = resnet50(weights=ResNet50_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, 2)
model.load_state_dict(torch.load('/content/drive/MyDrive/Fake News Model/trained_image_model.pth'))
model.eval()

<ipython-input-22-c1b403ccb078>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/drive/MyDrive/Fake News Model/trained_image_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

*To Save the Model*

In [ ]:
#Save the Trained_Model
torch.save(resnet.state_dict(), "trained_image_model.pth")

torch.save(model.state_dict(), '/content/drive/MyDrive/Fake News Model/trained_image_model.pth')


In [ ]:
model = resnet50(weights=ResNet50_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, 2)

model.load_state_dict(torch.load('/content/drive/MyDrive/Fake News Model/trained_image_model.pth'))
model.eval()


<ipython-input-23-9f5a684027fc>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/drive/MyDrive/Fake News Model/trained_image_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

*Test Image Model*

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {100 * correct / total:.2f}%")

Accuracy on test set: 98.61%


# ***Text Model***

In [ ]:
 fake_df = pd.read_csv('/content/drive/MyDrive/Fake News Model/text_training/Fake.csv')
 true_df = pd.read_csv('/content/drive/MyDrive/Fake News Model/text_training/True.csv')
 fake_df['label'] = 1
 true_df['label'] = 0
 df2 = pd.concat([fake_df, true_df], ignore_index=True)

  from sklearn.feature_extraction.text import TfidfVectorizer
 stop_words = set(stopwords.words('english'))

 #preprocessing
 def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text
 df2['content'] = (df2['title'] + " " + df2['text']).apply(clean_text)
 X_train, X_test, y_train, y_test = train_test_split(df2['content'], df2['label'], test_size=0.2, random_state=42)
 vectorizer = TfidfVectorizer(max_features=5000)
 X_train_tfidf = vectorizer.fit_transform(X_train)
 X_test_tfidf = vectorizer.transform(X_test)

 #Model Training
 from sklearn.linear_model import LogisticRegression
 from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

 model = LogisticRegression(max_iter=1000, random_state=42)
 model.fit(X_train_tfidf, y_train)
 y_pred = model.predict(X_test_tfidf)
 # Model Eval
 accuracy = accuracy_score(y_test, y_pred)
 print("Accuracy:", accuracy)
 print("\nClassification Report:\n", classification_report(y_test, y_pred))
 print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


In [ ]:
 import joblib
 model_path = '/content/drive/My Drive/Fake News Model/fake_news_text_model.pkl'
 joblib.dump(model, model_path)
 joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')

# ***Ensemble Learning***

In [ ]:
 def combine_embeddings(row):
    title_emb = row['title_embedding']
    desc_emb = row['description_embedding']
    image_emb = row['image_embedding']
    if image_emb is not None:
        return np.concatenate((title_emb, desc_emb, image_emb))
    else:
        return np.concatenate((title_emb, desc_emb))
 df['combined_embedding'] = df.apply(combine_embeddings, axis=1)
 from sklearn.ensemble import RandomForestClassifier
 from sklearn.linear_model import LogisticRegression
 random_forest = RandomForestClassifier(random_state=42)
 logistic_regression = LogisticRegression(max_iter=1000, random_state=42)

 #Stacking Classifier
 from sklearn.ensemble import StackingClassifier
 base_models = [
    ('random_forest', random_forest),
    ('logistic_regression', logistic_regression)
 ]
 stacking_clf = StackingClassifier(
    estimators=base_models,
    final_estimator=LogisticRegression(max_iter=1000, random_state=42)
 )




# New Section

In [ ]:

def combine_embeddings(row):
    title_emb = row['title_embedding']
    desc_emb = row['description_embedding']
    image_emb = row['image_embedding']

    if image_emb is not None:
        return np.concatenate((title_emb, desc_emb, image_emb))
    else:
        return np.concatenate((title_emb, desc_emb))

df['combined_embedding'] = df.apply(combine_embeddings, axis=1)

NameError: name 'df' is not defined

In [ ]:
image_model = resnet50(weights=ResNet50_Weights.DEFAULT)
image_model.fc = torch.nn.Linear(image_model.fc.in_features, 2)
image_model.load_state_dict(torch.load('/content/drive/MyDrive/Fake News Model/trained_image_model.pth'))
image_model.eval()

text_model = joblib.load('/content/drive/MyDrive/Fake News Model/fake_news_text_model.pkl')
vectorizer = joblib.load('tfidf_vectorizer.pkl')


NameError: name 'resnet50' is not defined

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from PIL import Image

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

image_predictions = []
for path in df['image_path']:
    if path is None or not os.path.exists(path):
        print(f"Warning: Image path {path} is invalid or does not exist.")
        image_predictions.append(None)
        continue

    image = Image.open(path)

    if image.mode != 'RGB':
        image = image.convert('RGB')

    image = transform(image).unsqueeze(0)

    with torch.no_grad():
        output = image_model(image)
        _, predicted = torch.max(output, 1)
        image_predictions.append(predicted.item())
image_predictions = [pred for pred in image_predictions if pred is not None]

In [ ]:
text_features = vectorizer.transform(df['processed_description'])
text_predictions = text_model.predict(text_features)


In [ ]:
import numpy as np
from scipy.stats import mode
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

valid_image_predictions = []
valid_text_predictions = []
missing_indices = []

for idx, (img_pred, txt_pred) in enumerate(zip(image_predictions, text_predictions)):
    if img_pred is not None:
        valid_image_predictions.append(img_pred)
        valid_text_predictions.append(txt_pred)
    else:
        missing_indices.append(idx)

valid_image_predictions = np.array(valid_image_predictions)
valid_text_predictions = np.array(valid_text_predictions)

if len(valid_image_predictions) == len(valid_text_predictions):
    combined_predictions = np.vstack((valid_image_predictions, valid_text_predictions)).T
    final_predictions, _ = mode(combined_predictions, axis=1)
    final_predictions = final_predictions.flatten()

    df['label'] = None

    for idx, prediction in enumerate(final_predictions):
        df.at[idx, 'label'] = prediction

    print(df[['title', 'processed_description', 'label']])

